In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('titanic.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

'''
Создание x и y множеств. Удалим имена пассажиров, 
так они вряд ли влюяют на что-то и категоризируем пол пассажиров.
'''
X = df.drop('Survived', axis = 1).drop('Name', axis = 1)
X['Sex'] = LabelEncoder().fit_transform(X['Sex'])

Y = df['Survived']
print("passagires data")
print(X.head())
print("Is survived?")
print(Y.head())

passagires data
   Pclass  Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard  \
0       3    1  22.0                        1                        0   
1       1    0  38.0                        1                        0   
2       3    0  26.0                        0                        0   
3       1    0  35.0                        1                        0   
4       3    1  35.0                        0                        0   

      Fare  
0   7.2500  
1  71.2833  
2   7.9250  
3  53.1000  
4   8.0500  
Is survived?
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [11]:
#ансамбль моделей типа бегинг (bagging)
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = BaggingClassifier(base_estimator=RandomForestClassifier(),
                        n_estimators=10, random_state=1).fit(X, Y)

sc_bag = cross_val_score(clf, X, Y, scoring='accuracy', cv = 5)
print(sc_bag)

[0.78651685 0.80898876 0.86440678 0.81355932 0.84745763]


In [16]:
#ансамбль моделей типа бустинг (boosting)
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(base_estimator=RandomForestClassifier(),
                        n_estimators=10, random_state=1).fit(X, Y)

sc_boost_a = cross_val_score(abc, X, Y, scoring='accuracy', cv = 5)
print(sc_boost_a)

[0.76404494 0.78651685 0.84180791 0.8079096  0.84180791]


In [21]:
#ансамбль моделей типа стекинг (stacking)
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier 

estimators = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gnb', GaussianNB())
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
clf.fit(X, Y)
sc_st = cross_val_score(clf, X, Y, scoring='accuracy', cv = 5)
print(sc_st)

[0.76966292 0.79775281 0.83615819 0.81355932 0.84180791]


In [24]:
#ансамбль моделей типа голосования (voting)
from sklearn.ensemble import VotingClassifier

m_vote_h = VotingClassifier(estimators=estimators, voting = 'hard').fit(X, Y)
sc_vh = cross_val_score(m_vote_h, X, Y, scoring='accuracy', cv = 5)
print(sc_vh)

m_vote_s = VotingClassifier(estimators=estimators, voting = 'soft').fit(X, Y)
sc_vs = cross_val_score(m_vote_s, X, Y, scoring='accuracy', cv = 5)
print(sc_vs)

[0.78089888 0.8258427  0.81355932 0.79661017 0.77966102]
[0.75280899 0.79775281 0.83050847 0.77966102 0.81920904]
